**1. Read dataset**

In [1]:
import os
import torch
import random
import warnings
import faiss
import numpy as np
import pandas as pd
import torch.nn as nn
from tqdm import tqdm
from torch import optim
from torch import nn
from copy import deepcopy
from sklearn import metrics
import torch.nn.functional as F
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn import manifold
from random import sample
from sklearn.model_selection import train_test_split
from collections import Counter, OrderedDict
from torch.utils.data import DataLoader, Dataset
from sklearn import metrics
from copy import deepcopy

import sys
sys.path.append('..')
sys.path.append('../..')

import torchvision.models as models

import pcl.loader
import pcl.builder2
import warnings

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
seed = 10

def setup_seed(seed=seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [4]:
f = open('output.txt', 'a')
f.write('seed: ' + str(seed) + '\n')
f.close()

In [5]:
logdata = pd.read_csv(r'./BGL.log_structured_v1.csv')[:500000]

In [6]:
def slide_window(logdata, window_size=20, step_size=10):
    logdata["Label"] = logdata["Label"].apply(lambda x: 0 if x == '-' else 1)
    data = logdata.loc[:, ['EventId', 'Label']]
    data['Key_label'] = data['Label']
    data.rename(columns={'Label': 'Sequence_label'})
    logkey = data['EventId']
    logkey_label = data['Key_label']

    new_data = []
    idx = 0

    while idx <= data.shape[0] - window_size:
        new_data.append([logkey[idx: idx + window_size].values,
                         max(logkey_label[idx: idx + window_size]),
                         logkey_label[idx: idx + window_size].values
                         ])
        idx += step_size
    return pd.DataFrame(new_data, columns=['EventId', 'Sequence_label', 'Key_label'])


dataset = slide_window(logdata)

In [7]:
normal_ds = dataset[dataset['Sequence_label']==0]
abnormal_ds = dataset[dataset['Sequence_label']==1]

setup_seed()

train_normal_ds = normal_ds.sample(n=1000, random_state=seed)
rest_normal_ds = normal_ds.drop(train_normal_ds.index)
val_normal_ds, test_normal_ds = train_test_split(rest_normal_ds, train_size=100, test_size=10000, random_state=seed)

train_abnormal_ds = abnormal_ds.sample(n=100, random_state=seed)
rest_abnormal_ds = abnormal_ds.drop(train_abnormal_ds.index)
val_abnormal_ds, test_abnormal_ds = train_test_split(rest_abnormal_ds, train_size=20, test_size=1000, random_state=seed)

train_ds = pd.concat([train_normal_ds, train_abnormal_ds])
val_ds = pd.concat([val_normal_ds, val_abnormal_ds])
test_ds = pd.concat([test_normal_ds, test_abnormal_ds])

**2. Preprocessing**

In [8]:
counts = Counter()

for index, row in train_ds.iterrows():
    counts.update(row['EventId'])

logkey2index = {"": 0, "UNK": 1}
logkeys = ["", "UNK"]

for word in counts:
    logkey2index[word] = len(logkeys)
    logkeys.append(word)

In [9]:
def encode_sequence(sequence, logkey2index):
    return np.array([logkey2index.get(logkey, logkey2index["UNK"]) for logkey in sequence])


train_ds.loc[:, 'Encoded'] = train_ds.loc[:, 'EventId'].apply(lambda x: encode_sequence(x, logkey2index))
val_ds.loc[:, 'Encoded'] = val_ds.loc[:, 'EventId'].apply(lambda x: encode_sequence(x, logkey2index))
test_ds.loc[:, 'Encoded'] = test_ds.loc[:, 'EventId'].apply(lambda x: encode_sequence(x, logkey2index))

In [10]:
batch_size_train = 20
batch_size_val = 20
batch_size_test = 100

In [11]:
setup_seed()

def compose_data_list(dataset):
    data_list = []
    for i in range(dataset.shape[0]):
        data_list.append([dataset.iloc[i, 1], dataset.iloc[i, 2], dataset.iloc[i, 3]])
    return data_list


train_data_list = compose_data_list(train_ds)
val_data_list = compose_data_list(val_ds)
test_data_list = compose_data_list(test_ds)


def collate_fn(data_list):
    sequence = torch.tensor([x[2] for x in data_list])
    sequence_label = torch.tensor([x[0] for x in data_list])
    key_label = torch.tensor([x[1] for x in data_list])
    return sequence, sequence_label, key_label


train_loader = DataLoader(train_data_list, batch_size=batch_size_train, collate_fn=collate_fn, shuffle=True)
val_loader = DataLoader(val_data_list, batch_size=batch_size_val, collate_fn=collate_fn, shuffle=True)
test_loader = DataLoader(test_data_list, batch_size=batch_size_test, collate_fn=collate_fn, shuffle=True)

In [12]:
setup_seed()

normal_key_bank = set(np.array([seq.tolist() for seq in train_normal_ds.EventId.tolist()]).reshape(-1).tolist())
abnormal_key_bank = []

for i in range(train_abnormal_ds.shape[0]):
    abnormal_key_bank += train_abnormal_ds.iloc[i, 0][train_abnormal_ds.iloc[i, 2]==1].tolist()
abnormal_key_bank = set(abnormal_key_bank)

normal_index_bank = [logkey2index.get(normal_key) for normal_key in normal_key_bank]
abnormal_index_bank = [logkey2index.get(abnormal_key) for abnormal_key in abnormal_key_bank]

def sequence_augmentation(sequence):
#     n = np.random.choice(10, 1) + 1
    n = 1
    r_position = list(np.random.choice(20, n))
    r_key = [normal_index_bank[k] for k in np.random.choice(len(normal_index_bank), n)]
    for r in range(len(r_position)):
        sequence[r_position[r]] = r_key[r]
    return sequence
    
def compose_pairs(dataset):
    normal_ds = dataset[dataset['Sequence_label'] == 0]
    abnormal_ds = dataset[dataset['Sequence_label'] == 1]
    
    normal_pairs = []
    abnormal_pairs = []

    normal_sequence_bank = [x.tolist() for x in normal_ds.Encoded.tolist()]

    for i in range(normal_ds.shape[0]):
        a, b = np.random.choice(range(0, normal_ds.shape[0]), 2)
        normal_pairs.append([normal_sequence_bank[a], normal_sequence_bank[b]])

    normal_ds['Pair0'] = pd.Series([x[0] for x in normal_pairs]).values
    normal_ds['Pair1'] = pd.Series([x[1] for x in normal_pairs]).values
        
    for j in range(abnormal_ds.shape[0]):
        sequence = list(abnormal_ds.iloc[j, 3])
        seq0 = deepcopy(sequence)
        seq1 = deepcopy(sequence)
        pair0 = sequence_augmentation(seq0)
        pair1 = sequence_augmentation(seq1)
        abnormal_pairs.append([pair0, pair1])   

    abnormal_ds['Pair0'] = pd.Series([x[0] for x in abnormal_pairs]).values
    abnormal_ds['Pair1'] = pd.Series([x[1] for x in abnormal_pairs]).values

    return pd.concat([normal_ds, abnormal_ds])    

with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    train_ds2 = compose_pairs(train_ds)
    val_ds2 = compose_pairs(val_ds)
    test_ds2 = compose_pairs(test_ds)

    train_ds2['Index'] = range(train_ds2.shape[0])
    val_ds2['Index'] = range(val_ds2.shape[0])
    test_ds2['Index'] = range(test_ds2.shape[0])

In [13]:
def compose_data_list2(dataset):
    data_list = []
    for i in range(dataset.shape[0]):
        data_list.append([dataset.iloc[i, 6], dataset.iloc[i, 3], [dataset.iloc[i, 4], 
                          dataset.iloc[i, 5]], dataset.iloc[i, 1], dataset.iloc[i, 2]])
    return data_list


train_data_list2 = compose_data_list2(train_ds2)
val_data_list2 = compose_data_list2(val_ds2)
test_data_list2 = compose_data_list2(test_ds2)


def collate_fn2(data_list):
    index = torch.tensor([x[0] for x in data_list])
    sequence = torch.tensor([x[1] for x in data_list])
    sequence_pair = torch.tensor([x[2] for x in data_list])
    sequence_label = torch.tensor([x[3] for x in data_list])
    key_label = torch.tensor([x[4] for x in data_list])
    return index, sequence, sequence_pair, sequence_label, key_label


train_loader = DataLoader(train_data_list2, batch_size=batch_size_train, collate_fn=collate_fn2, shuffle=True)
val_loader = DataLoader(val_data_list2, batch_size=batch_size_val, collate_fn=collate_fn2, shuffle=True)
test_loader = DataLoader(test_data_list2, batch_size=batch_size_test, collate_fn=collate_fn2, shuffle=True)

**3. Model**

In [14]:
vocab_size = len(logkeys)
embedding_dim = 100
hidden_dim = 256
num_layers = 1
epochs = 150
num_cluster_n = ['6']
num_cluster_a = ['5']

f = open('output.txt', 'a')
f.write('Abnormal clusters: ' + num_cluster_a[0] + '\n')
f.close()

num_cluster = [str(int(num_cluster_n[i]) + int(num_cluster_a[i])) for i in range(len(num_cluster_n))]

In [15]:
class Encoder(nn.Module):
    def __init__(self, vocab_size=len(logkeys), embedding_dim=embedding_dim, hidden_dim=hidden_dim,
                 num_layers=num_layers, num_classes=2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(input_size=embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=num_layers,
                            batch_first=True,
                            bias=True)

    def forward(self, x):
        embedded = self.embeddings(x)
        out, (hidden, cell) = self.lstm(embedded)
        return nn.functional.normalize(torch.mean(out, dim=1), dim=1)

In [16]:
setup_seed()

model = pcl.builder2.MoCo(Encoder,hidden_dim, 100, 0.999, 0.05).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), 0.001, weight_decay=1e-6)

**4. Training**

In [17]:
def compute_features(train_ds, model):
    model.eval()
    encoded_sequences = torch.tensor(train_ds.Encoded.tolist()).to(device)
    features = model(encoded_sequences, is_eval=True)   
    return features.cpu().numpy()

In [18]:
def run_kmeans(x, num_cluster, temperature):
    """
    Args:
        x: data to be clustered
    """

    results = {'im2cluster': [], 'centroids': [], 'density': []}

    # intialize faiss clustering parameters
    d = x.shape[1]
    k = int(num_cluster[0])
    clus = faiss.Clustering(d, k)
    clus.verbose = True
    clus.niter = 10
    clus.nredo = 5
    clus.seed = seed
    clus.max_points_per_centroid = 1000
    clus.min_points_per_centroid = 10

    res = faiss.StandardGpuResources()
    cfg = faiss.GpuIndexFlatConfig()
    cfg.useFloat16 = False
    cfg.device = 0
    index = faiss.GpuIndexFlatL2(res, d, cfg)

    clus.train(x, index)

    D, I = index.search(x, 3)  # for each sample, find cluster distance and assignments
    im2cluster = [int(n[0]) for n in I]
    # get cluster centroids
    centroids = faiss.vector_to_array(clus.centroids).reshape(k, d)

    # sample-to-centroid distances for each cluster 
    Dcluster = [[] for c in range(k)]
    for im, i in enumerate(im2cluster):
        Dcluster[i].append(D[im][0])

    # concentration estimation (phi)        
    density = np.zeros(k)
    for i, dist in enumerate(Dcluster):
        if len(dist) > 1:
#             d = (np.asarray(dist) ** 0.5).mean() / np.log(len(dist) + 10)
            d = (np.asarray(dist) ** 0.5).mean() / np.log(len(dist) + 10)
            density[i] = d

            # if cluster only has one point, use the max to estimate its concentration
    dmax = density.max()
    for i, dist in enumerate(Dcluster):
        if len(dist) <= 1:
            density[i] = dmax

    density = density.clip(np.percentile(density, 10),
                           np.percentile(density, 90))  # clamp extreme values for stability
    density = temperature * density / density.mean()  # scale the mean to temperature

    # convert to cuda Tensors for broadcast
    centroids = torch.Tensor(centroids).cuda()
    centroids = nn.functional.normalize(centroids, p=2, dim=1)

    im2cluster = torch.LongTensor(im2cluster).cuda()
    density = torch.Tensor(density).cuda()

    results['centroids'] = centroids
    results['density'] = density
    results['im2cluster'] = im2cluster

    return results

In [19]:
def train_model(train_loader, model, criterion, optimizer, cluster_result=None):
    model.train()
    
    acc_epoch = []
    loss_epoch = []
    for index, sequence, sequence_pair, sequence_label, key_label in train_loader:
        pair0 = sequence_pair[:, 0, :].to(device)
        pair1 = sequence_pair[:, 1, :].to(device)
        sequence_label = sequence_label.to(device)

        output, target, output_proto, target_proto = model(im_q=pair0, im_k=pair1, cluster_result=cluster_result, 
                                                           index = index, sequence_label=sequence_label)
        info_loss = criterion(output, target)
        # ProtoNCE loss
        if output_proto is not None:
            loss_proto = 0
            for proto_out,proto_target in zip(output_proto, target_proto):
                loss_proto += criterion(proto_out, proto_target)

            # average loss across all sets of prototypes
            loss_proto /= len(num_cluster)
            loss = info_loss + loss_proto 
#             print(info_loss.item(), loss_proto.item())

        # losses.update(loss.item(), images[0].size(0))
        loss_epoch.append(loss.item())

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0, norm_type=2)
        optimizer.step()
        
        return np.mean(loss_epoch)

In [20]:
warnings.filterwarnings('ignore')
torch.autograd.set_detect_anomaly(True)

setup_seed()

best_val_acc_sequence = -1000
best_val_model = None

for i in range(epochs):    
    train_normal = train_ds2[train_ds['Sequence_label']==0]
    train_abnormal = train_ds2[train_ds['Sequence_label']==1]
 
    
    features_n = compute_features(train_normal, model)  
    cluster_result_n = run_kmeans(features_n, num_cluster_n, 0.05)
    
    features_a = compute_features(train_abnormal, model)
    cluster_result_a = run_kmeans(features_a, num_cluster_a, 0.05)

    cluster_result_a['im2cluster'] = cluster_result_a['im2cluster'] + int(num_cluster_n[0])

    keys = ['im2cluster', 'centroids', 'density']
    cluster_result = {'im2cluster': [[], [], []], 'centroids': [[], [], []], 'density': [[], [], []]}

    for key in keys:
        cluster_result[key] = torch.cat((cluster_result_n[key], cluster_result_a[key]), 0)
    epoch_loss = train_model(train_loader, model, criterion, optimizer, cluster_result)
    
#     model.eval()
    
#     correct_sequence = 0

#     faiss_search = faiss.IndexFlatL2(cluster_result['centroids'].size(1))
#     faiss_search.add(cluster_result['centroids'].detach().cpu().numpy())
    
#     with torch.no_grad():
#         for index, sequence, sequence_pair, sequence_label, key_label in train_loader:
#             true_label = []
#             pred_label = []
#             sequence = sequence.to(device)

#             true_label += sequence_label
#             sq = model.encoder_k(sequence).detach().cpu().numpy()
#             D, I = faiss_search.search(sq, 1)
#             pred_label += [1 if pred>int(num_cluster_n[0]) else 0 for pred in I]

#             acc_sequence = (torch.tensor(true_label)==torch.tensor(pred_label)).sum().item()
#             correct_sequence += acc_sequence
            
#     if correct_sequence > best_val_acc_sequence:
#         best_val_acc_sequence = correct_sequence
#         best_val_model = deepcopy(model.state_dict())
    
#     print(f'Epoch {i}: {epoch_loss}')

Clustering 1000 points in 256D to 6 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.01 s, search 0.00 s): objective=103.742 imbalance=2.667 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=132.398 imbalance=2.623 nsplit=0       
Outer iteration 2 / 5
  Iteration 9 (0.02 s, search 0.01 s): objective=145.039 imbalance=2.588 nsplit=0       
Outer iteration 3 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=138.627 imbalance=2.642 nsplit=0       
Outer iteration 4 / 5
  Iteration 9 (0.03 s, search 0.02 s): objective=148.986 imbalance=2.620 nsplit=0       
Clustering 100 points in 256D to 5 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.00 s, search 0.00 s): objective=2.126 imbalance=2.665 nsplit=1       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.00 s, search 0.00 s): 

Clustering 1000 points in 256D to 6 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.01 s, search 0.00 s): objective=103.742 imbalance=2.667 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=132.398 imbalance=2.623 nsplit=0       
Outer iteration 2 / 5
  Iteration 9 (0.02 s, search 0.01 s): objective=145.039 imbalance=2.588 nsplit=0       
Outer iteration 3 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=138.627 imbalance=2.642 nsplit=0       
Outer iteration 4 / 5
  Iteration 9 (0.03 s, search 0.02 s): objective=148.986 imbalance=2.620 nsplit=0       
Clustering 100 points in 256D to 5 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.00 s, search 0.00 s): objective=2.126 imbalance=2.665 nsplit=1       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.00 s, search 0.00 s): 

Clustering 1000 points in 256D to 6 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.01 s, search 0.00 s): objective=103.742 imbalance=2.667 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=132.398 imbalance=2.623 nsplit=0       
Outer iteration 2 / 5
  Iteration 9 (0.02 s, search 0.01 s): objective=142.853 imbalance=2.601 nsplit=0       
Outer iteration 3 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=138.625 imbalance=2.642 nsplit=0       
Outer iteration 4 / 5
  Iteration 9 (0.03 s, search 0.02 s): objective=148.986 imbalance=2.620 nsplit=0       
Clustering 100 points in 256D to 5 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.00 s, search 0.00 s): objective=2.12602 imbalance=2.665 nsplit=1       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.00 s, search 0.00 s)

Clustering 1000 points in 256D to 6 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.01 s, search 0.00 s): objective=103.742 imbalance=2.667 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=132.399 imbalance=2.623 nsplit=0       
Outer iteration 2 / 5
  Iteration 9 (0.02 s, search 0.01 s): objective=142.855 imbalance=2.601 nsplit=0       
Outer iteration 3 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=138.624 imbalance=2.642 nsplit=0       
Outer iteration 4 / 5
  Iteration 9 (0.03 s, search 0.02 s): objective=148.988 imbalance=2.620 nsplit=0       
Clustering 100 points in 256D to 5 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.00 s, search 0.00 s): objective=2.12604 imbalance=2.665 nsplit=1       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.00 s, search 0.00 s)

Clustering 1000 points in 256D to 6 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.01 s, search 0.00 s): objective=103.743 imbalance=2.667 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=132.4 imbalance=2.623 nsplit=0       
Outer iteration 2 / 5
  Iteration 9 (0.02 s, search 0.01 s): objective=145.043 imbalance=2.588 nsplit=0       
Outer iteration 3 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=138.628 imbalance=2.642 nsplit=0       
Outer iteration 4 / 5
  Iteration 9 (0.03 s, search 0.02 s): objective=148.993 imbalance=2.620 nsplit=0       
Clustering 100 points in 256D to 5 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.00 s, search 0.00 s): objective=2.12606 imbalance=2.665 nsplit=1       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.00 s, search 0.00 s): 

...

Clustering 1000 points in 256D to 6 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.01 s, search 0.00 s): objective=130.201 imbalance=2.611 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=139.462 imbalance=2.620 nsplit=0       
Outer iteration 2 / 5
  Iteration 9 (0.02 s, search 0.01 s): objective=166.204 imbalance=2.632 nsplit=0       
Outer iteration 3 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=172.625 imbalance=2.685 nsplit=0       
Outer iteration 4 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=215.304 imbalance=2.776 nsplit=0       
Clustering 100 points in 256D to 5 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.00 s, search 0.00 s): objective=2.00179 imbalance=2.665 nsplit=1       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.00 s, search 0.00 s)

Clustering 1000 points in 256D to 6 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.01 s, search 0.00 s): objective=130.151 imbalance=2.611 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=139.461 imbalance=2.620 nsplit=0       
Outer iteration 2 / 5
  Iteration 9 (0.02 s, search 0.01 s): objective=166.16 imbalance=2.632 nsplit=0       
Outer iteration 3 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=172.577 imbalance=2.685 nsplit=0       
Outer iteration 4 / 5
  Iteration 9 (0.03 s, search 0.02 s): objective=215.241 imbalance=2.776 nsplit=0       
Clustering 100 points in 256D to 5 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.00 s, search 0.00 s): objective=2.00127 imbalance=2.665 nsplit=1       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.00 s, search 0.00 s):

Clustering 1000 points in 256D to 6 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.01 s, search 0.00 s): objective=130.106 imbalance=2.611 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=139.459 imbalance=2.620 nsplit=0       
Outer iteration 2 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=166.12 imbalance=2.632 nsplit=0       
Outer iteration 3 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=127.766 imbalance=2.586 nsplit=0       
Objective improved: keep new clusters
Outer iteration 4 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=201.447 imbalance=2.703 nsplit=0       
Clustering 100 points in 256D to 5 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.00 s, search 0.00 s): objective=2.0007 imbalance=2.665 nsplit=1       
Objective improved: keep new clusters
Outer iteration 1 / 5
 

Clustering 1000 points in 256D to 6 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.01 s, search 0.00 s): objective=130.053 imbalance=2.611 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=139.457 imbalance=2.620 nsplit=0       
Outer iteration 2 / 5
  Iteration 9 (0.02 s, search 0.01 s): objective=166.071 imbalance=2.632 nsplit=0       
Outer iteration 3 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=127.723 imbalance=2.586 nsplit=0       
Objective improved: keep new clusters
Outer iteration 4 / 5
  Iteration 9 (0.03 s, search 0.02 s): objective=215.121 imbalance=2.776 nsplit=0       
Clustering 100 points in 256D to 5 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.00 s, search 0.00 s): objective=2.00021 imbalance=2.665 nsplit=1       
Objective improved: keep new clusters
Outer iteration 1 / 5

Clustering 1000 points in 256D to 6 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.01 s, search 0.00 s): objective=130.002 imbalance=2.611 nsplit=0       
Objective improved: keep new clusters
Outer iteration 1 / 5
  Iteration 9 (0.01 s, search 0.01 s): objective=139.455 imbalance=2.620 nsplit=0       
Outer iteration 2 / 5
  Iteration 9 (0.02 s, search 0.01 s): objective=166.025 imbalance=2.632 nsplit=0       
Outer iteration 3 / 5
  Iteration 9 (0.02 s, search 0.02 s): objective=127.681 imbalance=2.586 nsplit=0       
Objective improved: keep new clusters
Outer iteration 4 / 5
  Iteration 9 (0.03 s, search 0.02 s): objective=215.055 imbalance=2.776 nsplit=0       
Clustering 100 points in 256D to 5 clusters, redo 5 times, 10 iterations
  Preprocessing in 0.00 s
Outer iteration 0 / 5
  Iteration 9 (0.00 s, search 0.00 s): objective=1.99964 imbalance=2.665 nsplit=1       
Objective improved: keep new clusters
Outer iteration 1 / 5

In [21]:
cluster_sequences = [[] for i in range(int(num_cluster[0]))]
cluster_labels = [[] for i in range(int(num_cluster[0]))]

for i in range(len(cluster_result['im2cluster'])):
    cluster_sequences[cluster_result['im2cluster'][i]].append(i)

for i in range(len(cluster_sequences)):
    cluster = cluster_sequences[i]
    
    for sequence_index in cluster:
        cluster_labels[i].append(train_data_list2[sequence_index][3])  

In [22]:
model.eval()
# model.load_state_dict(best_val_model)

faiss_search = faiss.IndexFlatL2(cluster_result['centroids'].size(1))
faiss_search.add(cluster_result['centroids'].detach().cpu().numpy())

true_label = []
pred_label = []

for index, sequence, sequence_pair, sequence_label, key_label in test_loader:
    sequence = sequence.to(device)
    
    true_label += sequence_label.tolist()
    sq = nn.functional.normalize(model.encoder_k(sequence), dim=1).detach().cpu().numpy()
    D, I = faiss_search.search(sq, 1)
    pred_label += [1 if pred>int(num_cluster_n[0])-1 else 0 for pred in I]

In [23]:
print(metrics.classification_report(true_label, pred_label, digits=4))
print(metrics.confusion_matrix(true_label, pred_label))

fpr, tpr, thresholds = metrics.roc_curve(true_label, pred_label, pos_label=1)
print(metrics.auc(fpr, tpr))

f = open('output.txt', 'a')
f.write('Anomaly detection on testing sequences:'+'\n')
f.write(str(metrics.classification_report(true_label, pred_label, digits=4))+'\n')
f.write(str(metrics.confusion_matrix(true_label, pred_label))+'\n')
f.write(str(metrics.auc(fpr, tpr))+'\n')
f.write('-'*50 + '\n')
f.close()

              precision    recall  f1-score   support

           0     0.9947    0.9948    0.9948     10000
           1     0.9479    0.9470    0.9475      1000

    accuracy                         0.9905     11000
   macro avg     0.9713    0.9709    0.9711     11000
weighted avg     0.9905    0.9905    0.9905     11000

[[9948   52]
 [  53  947]]
0.9709
